# Lab | Customer Analysis Round 6
For this lab, we still keep using the `marketing_customer_analysis.csv` file that you can find in the `files_for_lab` folder.

### Get the data
We are using the `marketing_customer_analysis.csv` file.

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import norm
from scipy import stats #para box-cox entre otros
from scipy.stats import skew
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

### Dealing with the data

Already done in the round 2.

In [18]:
# Cargamos el archivo ya limpio

df= pd.read_csv('marketing_customer_analysis(2).csv')

### EDA (Exploratory Data Analysis) - Complete the following 

Done in the round 3.

In [19]:
df_2= df.copy()
df_2.columns

Index(['unnamed', 'customer', 'state', 'customer lifetime value', 'response',
       'coverage', 'education', 'effective to date', 'employmentstatus',
       'gender', 'income', 'location code', 'marital status',
       'monthly premium auto', 'months since last claim',
       'months since policy inception', 'number of open complaints',
       'number of policies', 'policy type', 'policy', 'renew offer type',
       'sales channel', 'total claim amount', 'vehicle class', 'vehicle size',
       'vehicle type', 'Month'],
      dtype='object')

In [20]:
df_2.shape

(10910, 27)

In [21]:
# Renombrar la columna 'unnamed: 0' a 'unnamed'
df_2.rename(columns={'unnamed: 0': 'unnamed'}, inplace=True)
df_2.info

<bound method DataFrame.info of        unnamed customer       state  customer lifetime value response  \
0            0  DK49336     Arizona              4809.216960       No   
1            1  KX64629  California              2228.525238       No   
2            2  LZ68649  Washington             14947.917300       No   
3            3  XL78013      Oregon             22332.439460      Yes   
4            4  QA50777      Oregon              9025.067525       No   
...        ...      ...         ...                      ...      ...   
10905    10905  FE99816      Nevada             15563.369440       No   
10906    10906  KX53892      Oregon              5259.444853       No   
10907    10907  TL39050     Arizona             23893.304100       No   
10908    10908  WA60547  California             11971.977650       No   
10909    10909  IV32877      Unknow              6857.519928   Unknow   

       coverage education effective to date employmentstatus gender  ...  \
0         Basic

In [22]:
# Visualizamos la columnas que son numericas
df_2._get_numeric_data()

,unnamed,customer lifetime value,income,monthly premium auto,months since last claim,months since policy inception,number of open complaints,number of policies,total claim amount,Month
0,0,4809.216960,48029,61,7.000000,52,0.000000,9,292.800000,2
1,1,2228.525238,0,64,3.000000,26,0.000000,1,744.924331,1
2,2,14947.917300,22139,100,34.000000,31,0.000000,2,480.000000,2
3,3,22332.439460,49078,97,10.000000,3,0.000000,2,484.013411,1
4,4,9025.067525,23675,117,15.149071,31,0.384256,7,707.925645,1
...,...,...,...,...,...,...,...,...,...,...
10905,10905,15563.369440,0,253,15.149071,40,0.384256,7,1214.400000,1
10906,10906,5259.444853,61146,65,7.000000,68,0.000000,6,273.018929,1
10907,10907,23893.304100,39837,201,11.000000,63,0.000000,2,381.306996,2
10908,10908,11971.977650,64195,158,0.000000,27,4.000000,6,618.288849,2


In [23]:
df_2.describe

<bound method NDFrame.describe of        unnamed customer       state  customer lifetime value response  \
0            0  DK49336     Arizona              4809.216960       No   
1            1  KX64629  California              2228.525238       No   
2            2  LZ68649  Washington             14947.917300       No   
3            3  XL78013      Oregon             22332.439460      Yes   
4            4  QA50777      Oregon              9025.067525       No   
...        ...      ...         ...                      ...      ...   
10905    10905  FE99816      Nevada             15563.369440       No   
10906    10906  KX53892      Oregon              5259.444853       No   
10907    10907  TL39050     Arizona             23893.304100       No   
10908    10908  WA60547  California             11971.977650       No   
10909    10909  IV32877      Unknow              6857.519928   Unknow   

       coverage education effective to date employmentstatus gender  ...  \
0         Bas

In [24]:
# Seleccionar todas las columnas categóricas
df_2_object= df_2.select_dtypes(object)
df_2_numerical= df_2.select_dtypes(np.number)

1.7 DATA PREPROCESSING

In [25]:
df_2_numerical

,unnamed,customer lifetime value,income,monthly premium auto,months since last claim,months since policy inception,number of open complaints,number of policies,total claim amount,Month
0,0,4809.216960,48029,61,7.000000,52,0.000000,9,292.800000,2
1,1,2228.525238,0,64,3.000000,26,0.000000,1,744.924331,1
2,2,14947.917300,22139,100,34.000000,31,0.000000,2,480.000000,2
3,3,22332.439460,49078,97,10.000000,3,0.000000,2,484.013411,1
4,4,9025.067525,23675,117,15.149071,31,0.384256,7,707.925645,1
...,...,...,...,...,...,...,...,...,...,...
10905,10905,15563.369440,0,253,15.149071,40,0.384256,7,1214.400000,1
10906,10906,5259.444853,61146,65,7.000000,68,0.000000,6,273.018929,1
10907,10907,23893.304100,39837,201,11.000000,63,0.000000,2,381.306996,2
10908,10908,11971.977650,64195,158,0.000000,27,4.000000,6,618.288849,2


In [26]:
df_2_numerical.head()

,unnamed,customer lifetime value,income,monthly premium auto,months since last claim,months since policy inception,number of open complaints,number of policies,total claim amount,Month
0,0,4809.216960,48029,61,7.000000,52,0.000000,9,292.800000,2
1,1,2228.525238,0,64,3.000000,26,0.000000,1,744.924331,1
2,2,14947.917300,22139,100,34.000000,31,0.000000,2,480.000000,2
3,3,22332.439460,49078,97,10.000000,3,0.000000,2,484.013411,1
4,4,9025.067525,23675,117,15.149071,31,0.384256,7,707.925645,1


In [27]:
#OLS Regression Results  

Y=df_2_numerical['total claim amount']
X=df_2_numerical.drop(['total claim amount'], axis=1)
x=sm.add_constant(X)
model=sm.OLS(Y,X).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:     total claim amount   R-squared (uncentered):                   0.850
Model:                            OLS   Adj. R-squared (uncentered):              0.850
Method:                 Least Squares   F-statistic:                              6877.
Date:                Tue, 16 May 2023   Prob (F-statistic):                        0.00
Time:                        19:06:38   Log-Likelihood:                         -73434.
No. Observations:               10910   AIC:                                  1.469e+05
Df Residuals:                   10901   BIC:                                  1.470e+05
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------

In [32]:
#Normalizando 

transformer=StandardScaler().fit(df_2_numerical)
x_normalized=transformer.transform(df_2_numerical)
print(x_normalized.shape)
pd.DataFrame(x_normalized)

(10910, 10)


,0,1,2,3,4,5,6,7,8,9
0,-1.731892,-0.466105,0.345635,-0.934819,-0.832977,0.139877,-4.339189e-01,2.509455,-0.486325,1.068914
1,-1.731575,-0.840946,-1.236462,-0.847713,-1.241846,-0.790709,-4.339189e-01,-0.824922,1.061160,-0.935529
2,-1.731257,1.006523,-0.507194,0.197554,1.926893,-0.611750,-4.339189e-01,-0.408125,0.154404,1.068914
3,-1.730940,2.079112,0.380189,0.110448,-0.526325,-1.613919,-4.339189e-01,-0.408125,0.168140,-0.935529
4,-1.730622,0.146240,-0.456597,0.691152,0.000000,-0.611750,-6.268564e-17,1.675860,0.934524,-0.935529
...,...,...,...,...,...,...,...,...,...,...
10905,1.730622,1.095916,-1.236462,4.639940,0.000000,-0.289625,-6.268564e-17,1.675860,2.668033,-0.935529
10906,1.730940,-0.400710,0.777715,-0.818678,-0.832977,0.712545,-4.339189e-01,1.259063,-0.554030,-0.935529
10907,1.731257,2.305825,0.075787,3.130109,-0.424107,0.533586,-4.339189e-01,-0.408125,-0.183393,1.068914
10908,1.731575,0.574273,0.878150,1.881595,-1.548498,-0.754917,4.083057e+00,1.259063,0.627725,1.068914


In [40]:
#One Hot/Label Encoding (categorical).

df_2_object2= df_2_object.drop('effective to date', axis=1)
print(df_2_object2)


      customer       state response  coverage education employmentstatus  \
0      DK49336     Arizona       No     Basic   College         Employed   
1      KX64629  California       No     Basic   College       Unemployed   
2      LZ68649  Washington       No     Basic  Bachelor         Employed   
3      XL78013      Oregon      Yes  Extended   College         Employed   
4      QA50777      Oregon       No   Premium  Bachelor    Medical Leave   
...        ...         ...      ...       ...       ...              ...   
10905  FE99816      Nevada       No   Premium  Bachelor       Unemployed   
10906  KX53892      Oregon       No     Basic   College         Employed   
10907  TL39050     Arizona       No  Extended  Bachelor         Employed   
10908  WA60547  California       No   Premium   College         Employed   
10909  IV32877      Unknow   Unknow     Basic  Bachelor       Unemployed   

      gender location code marital status     policy type        policy  \
0          M

In [42]:
#para sacar dummies vamos a usar la liberería OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

encoder=OneHotEncoder(handle_unknown='error', drop="first").fit(df_2_object2)
encoder.categories_
encoded=encoder.transform(df_2_object2).toarray()
encoded

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [47]:
datatest= df_2_object2.copy()

# Create an instance of the OneHotEncoder with drop='first' to drop the first column of each dummy variable
encoder = OneHotEncoder(handle_unknown='error', drop='first')

# Fit the encoder to the categorical data
encoder.fit(df_2_object2)

# Transform the categorical data into dummy variables
encoded = encoder.transform(df_2_object2).toarray()

# Get the feature names of the encoded variables
feature_names = encoder.get_feature_names_out(df_2_object2.columns)

# Convert the encoded array to a DataFrame with column names
X_cat_encoded = pd.DataFrame(encoded, columns=feature_names)

# Drop the original categorical columns from the original DataFrame
datatest = datatest.drop(df_2_object2.columns, axis=1)

# Concatenate the encoded DataFrame with the original DataFrame
datatest = pd.concat([datatest, X_cat_encoded], axis=1)

# Visualization of code
datatest

,customer_AA11235,customer_AA16582,customer_AA30683,customer_AA34092,customer_AA35519,customer_AA56476,customer_AA69265,customer_AA71604,customer_AA93585,customer_AB13432,...,vehicle class_Luxury Car,vehicle class_Luxury SUV,vehicle class_SUV,vehicle class_Sports Car,vehicle class_Two-Door Car,vehicle class_Unknow,vehicle size_Medsize,vehicle size_Small,vehicle size_Unknow,vehicle type_Unknow
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
10908,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
